### Development of a score based on the gaussian heatmaps 
This can be used to generate the 'ground truth' score of the heatmaps produced from the Contextual layer , which will be compared with the score produced from the FCN heatmaps layer. 

- First we generate the heatmaps, and also visually cehck them. 
- the we pass the heatmaps to the routine that prodcues the scores 


## Build MRCNN model

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, math, io, time, gc, argparse, platform, pprint
import numpy as np
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')
# import mrcnn.model_mrcnn  as mrcnn_modellib
# import mrcnn.model_fcn    as fcn_modellib
import mrcnn.visualize    as visualize
# import mrcnn.new_shapes   as shapes
import mrcnn.utils        as utils

from datetime            import datetime   
from mrcnn.utils         import command_line_parser, Paths
from mrcnn.coco          import prep_coco_dataset
from mrcnn.prep_notebook import build_fcn_training_pipeline,run_fcn_training_pipeline
from mrcnn.datagen  import data_gen_simulate

pp = pprint.PrettyPrinter(indent=2, width=100)
float_formatter = lambda x: "%10.4f" % x
np_format = {}
np_format['float']=float_formatter
# np.set_printoptions(linewidth=150, precision=3, floatmode='fixed', threshold =10000, formatter = np_format)  
np.set_printoptions(linewidth=190, precision=4, threshold=10000, suppress = True)

##------------------------------------------------------------------------------------
## Parse command line arguments
##------------------------------------------------------------------------------------
parser = command_line_parser()
input_parms  =" --batch_size 1 "
input_parms +=" --epochs 2 --steps_in_epoch 32  --last_epoch 0 "
input_parms +=" --lr 0.00001 --val_steps 8 " 
input_parms +=" --mrcnn_logs_dir train_mrcnn_coco "
input_parms +=" --fcn_logs_dir   train_fcn8_coco_adam "
input_parms +=" --mrcnn_model    last "
input_parms +=" --fcn_model      F:/models/train_fcn8_coco_adam/fcn20181109T0000/fcn_1569.h5"
input_parms +=" --opt            adagrad "
input_parms +=" --fcn_arch       fcn8 " 
input_parms +=" --fcn_layers     all " 
input_parms +=" --sysout        screen "
# input_parms +=" --coco_classes   62 63 67 78 79 80 81 82 72 73 74 75 76 77 "
input_parms +=" --coco_classes   78 79 80 81 82 44 46 47 48 49 50 51 34 35 36 37 38 39 40 41 42 43 10 11 13 14 15 "
input_parms +=" --new_log_folder    "
print(input_parms)

args = parser.parse_args(input_parms.split())
utils.display_input_parms(args)

mrcnn_model, fcn_model = build_fcn_training_pipeline(args = args, verbose = 0)

## Build Training & Validation Datasets

In [ ]:
load_class_ids = args.coco_classes
print('load coco classes: ', load_class_ids)

dataset_train, train_generator= prep_coco_dataset(['train', 'val35k'], fcn_model.config, generator = True,shuffle = False, load_coco_classes=load_class_ids)
# dataset_val, val_generator    = prep_coco_dataset(['minival'], fcn_config, generator = True, shuffle = False)
# dataset_test.display_class_info()
dataset_train.display_active_class_info()
print("Testing Dataset Image Count: {}".format(len(dataset_train.image_ids)))
print("Testing Dataset Class Count: {}".format(dataset_train.num_classes))
class_names = dataset_train.class_names


## Display Images

###  Display image with Ground Truth bounding boxes and masks

In [ ]:
train_batch_x, train_batch_y = next(train_generator)
visualize.display_training_batch(dataset_train, train_batch_x)

In [ ]:
IMAGE_LIST = train_batch_x[1][:,0]
print('IMAGE_LIST : ', IMAGE_LIST)
for image_id in IMAGE_LIST:
    print()
    print('IMAGE_ID : ', image_id)#### Load a specific image using image_id
    annotations = dataset_train.image_info[image_id]["annotations"]
#     print(annotations)
    for annotation in annotations:
        class_id = dataset_train.map_source_class_id( "coco.{}".format(annotation['category_id']))
        print("coco.id: {} --> class_id : {}  - {} ".format(annotation['category_id'],class_id, dataset_train.class_names[class_id]))

### Load a specific image using image_id

In [ ]:
train_batch_x, _ =  data_gen_simulate(dataset_train, mrcnn_model.config, [7143]) ## [1234])
visualize.display_training_batch(dataset_train, train_batch_x)


## Pass data through MRCNN-FCN model

####  `run_mrcnn_training_pipeline()`

In [ ]:
from mrcnn.prep_notebook import run_full_training_pipeline, run_mrcnn_training_pipeline, run_fcn_training_pipeline   ###run_pipeline_on_input
outputs = run_full_training_pipeline(mrcnn_model, fcn_model, dataset_train, train_batch_x, verbose= 1)


#### Load output data

In [ ]:
print(len(outputs), outputs.keys())
model_pr_heatmap                = outputs['mrcnn_output'][0]          # layer:  0   shape: (1, 256, 256, 81)
model_pr_heatmap_scores         = outputs['mrcnn_output'][1]          # layer:  1   shape: (1, 81, 200, 11)
model_gt_heatmap                = outputs['mrcnn_output'][2]          # layer:  2   shape: (1, 256, 256, 81)
model_gt_heatmap_scores         = outputs['mrcnn_output'][3]          # layer:  3   shape: (1, 81, 200, 11)
model_mrcnn_class               = outputs['mrcnn_output'][4]
model_mrcnn_bbox                = outputs['mrcnn_output'][5]
model_output_rois               = outputs['mrcnn_output'][6]
model_target_class_ids          = outputs['mrcnn_output'][7]
model_roi_gt_boxes              = outputs['mrcnn_output'][8]

model_fcn_heatmap               = outputs['fcn_output'][0]          # layer:  0   shape: (1, 256, 256, 81)
model_fcn_softmax               = outputs['fcn_output'][1]          # layer:  1   shape: ()
model_MSE_loss                  = outputs['fcn_output'][2]          # layer:  1   shape: ()
model_BCE_loss                  = outputs['fcn_output'][3]          # layer:  1   shape: ()
model_fcn_scores                = outputs['fcn_output'][4]

print('model_pr_heatmap       : ', model_gt_heatmap.shape)
print('model_pr_heatmap_scores: ', model_gt_heatmap_scores.shape)
print('model_gt_heatmap       : ', model_gt_heatmap.shape)
print('model_gt_heatmap_scores: ', model_gt_heatmap_scores.shape)
print('model_fcn_heatmap      : ', model_fcn_heatmap.shape)
print('model_fcn_softmax      : ', model_fcn_softmax.shape)
print('model_fcn_scores       : ', model_fcn_scores.shape)
print('model_CE_loss          : ', model_BCE_loss)
print('model_MSE_loss         : ', model_MSE_loss)

img_id = 0

#### Display output shapes

In [ ]:
print(outputs['image_batch'].shape)
for i in outputs:
    print('Group:', i)
    for item in outputs[i]:
        print(' Type ', type(item), item.shape)

#### Load input data

In [ ]:
# input_image      =  train_batch_x[0]
# input_image_meta =  train_batch_x[1]
# # input_rpn_match  =  train_batch_x[2]
# # input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = outputs['mrcnn_input'][4]
# # input_gt_bboxes    = train_batch_x[5]
# # input_gt_masks     = train_batch_x[6]
# print(' Input image shape is :', input_image.shape)
# h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]
# input_normlzd_gt_bboxes = tf.identity(input_gt_bboxes / [h,w,h,w])

# # gt_masks   =  train_batch_x[6]
# print(' input_rpn_match    ', input_rpn_match.shape)
# print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(input_gt_class_ids)
# print(' input_gt_bboxes    ', input_gt_bboxes.shape)
# print(' input_normlzd_gt_bboxes    ', input_normlzd_gt_bboxes.shape)
# print(input_image_meta)

### Display values from FCN Output

In [ ]:
float_formatter = lambda x: "%10.3f" % x
np_format = {}
np_format['float']=float_formatter
np.set_printoptions(linewidth=150, precision=3, floatmode='fixed', threshold =10000, formatter = np_format)

gt_class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
print('Image : {}  ClassIds: {}'.format(img_id, gt_class_ids))


In [ ]:
print(np.bincount(model_gt_heatmap_scores[img_id,:,:,4])) 

#### Display heatmap stats

In [ ]:
img_id = 0
gt_class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
pr_class_ids = np.unique(model_pr_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
fcn_class_ids = np.unique(model_fcn_scores[img_id,:,:,4]).astype(int).tolist()
print('Image : {}  GT ClassIds: {}'.format(img_id, gt_class_ids))
print('Image : {}  PR ClassIds: {}'.format(img_id, pr_class_ids))
print('Image : {}  FCN ClassIds: {}'.format(img_id, fcn_class_ids))
# for i in range(pr_class_ids[-1]+1):
for i in range(81):
    print('class:', i, '-', dataset_train.class_names[i],'predicted in MRCNN' if i in pr_class_ids else ' ' , ' (Ground Truth) ' if  i in gt_class_ids else ' ')
    print('FCN class: {:3d}   min: {:12.8f}    max: {:12.8f}    avg: {:12.8f}    sum: {:12.8f} '.format(i, np.min(model_fcn_heatmap[img_id,:,:,i]), 
                                                                    np.max(model_fcn_heatmap[img_id,:,:,i]), np.mean(model_fcn_heatmap[img_id,:,:,i]), 
                                                                    np.sum(model_fcn_heatmap[img_id,:,:,i])))
    print('PR  class: {:3d}   min: {:12.8f}    max: {:12.8f}    avg: {:12.8f}    sum: {:12.8f} '.format(i, np.min(model_pr_heatmap[img_id,:,:,i]), 
                                                                    np.max(model_pr_heatmap[img_id,:,:,i]),  np.mean(model_pr_heatmap[img_id,:,:,i]),
                                                                    np.sum(model_pr_heatmap[img_id,:,:,i])))
    print('GT  class: {:3d}   min: {:12.8f}    max: {:12.8f}    avg: {:12.8f}    sum: {:12.8f} '.format(i, np.min(model_gt_heatmap[img_id,:,:,i]), 
                                                                    np.max(model_gt_heatmap[img_id,:,:,i]),  np.mean(model_gt_heatmap[img_id,:,:,i]),
                                                                    np.sum(model_gt_heatmap[img_id,:,:,i])))
    print()

####  display fcn_scores

In [ ]:
np.set_printoptions(linewidth=190, precision=4, threshold=10000, suppress = True)
print('Image : {}  PR ClassIds: {}'.format(img_id, pr_class_ids))
print('Image : {}  fcn ClassIds: {}'.format(img_id, fcn_class_ids))
print(model_fcn_scores.shape)
for i in  fcn_class_ids:
    print('\nclass:', i, '-', dataset_train.class_names[i],' - predicted in MRCNN' if i in pr_class_ids else ' ' , ' - (Ground Truth) ' if  i in gt_class_ids else ' ')
    print(model_fcn_scores[img_id,i, :25])

In [ ]:
print(model_pr_heatmap_scores.shape)
print(model_fcn_scores[0,0:70])

In [ ]:
# for i in range(10):
#     print(model_fcn_heatmap[0,i,:,4])
#     print(np.min(model_fcn_heatmap[0,i,:,1]), np.max(model_fcn_heatmap[0,i,:,1]))

####  `Pred_Tensor`, `Pred_heatmap`, `mrcnn_class`

In [ ]:
np.set_printoptions(precision=4, threshold=None, linewidth=210, suppress=True)
img = 0

# print(KB.int_shape(output_rois))
# print(output_rois[img,:15]*[128, 128,128,128])
# print(input_gt_class_ids[0])

# print(' Pred_tensor')
# print(pred_tensor.shape)
# print(pred_tensor[img,:,:10])

# print(' gt_tensor')
# print(gt_tensor.shape)
# print(gt_tensor[img,:,:10])

# print(' output_rois')
# print(output_rois.shape)
# print(output_rois[img,:15] * [128, 128,128,128])


# print(' roi_gt_boxes')
# print(roi_gt_boxes.shape)
# print(roi_gt_boxes[img,:15] * [128, 128,128,128])

print(' Pred Heatmap Scores')
print(pred_heatmap_scores.dtype)


print(' FCN Scores')
print(fcn_scores.dtype)
for cls in range(4):
    print(pred_heatmap_scores[img,cls,:10])
    print(fcn_scores[img,cls,:10,2:])

# img = 2
# max_score = np.max(mrcnn_class, axis = -1)
# max_class = np.argmax(mrcnn_class, axis = -1)
# # print(' output_rois[',img,'] \n', output_rois[1]*[128,128,128,128])
# print('max class shape:',max_class.shape, 'max score shape: ',max_score.shape)
# print('max class[',img,']\n',max_class[img])
# print('max score[',img,']\n',max_score[img])
# print('mrcnn class.shape ',mrcnn_class.shape)
# print('mrcnn_class[',img,',:]\n',mrcnn_class[img,:])
# print(output_rois[1])
 


####  Display `output_rois` for visual check - passed on to  `build_pred_tensor()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('output_rois shape is ', output_rois.shape)
img = 0
for img in range(5):
    print('Image ', img , ' ------------')
    print(output_rois[img]*[128,128,128,128])


####  Display for visual check - `pred_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('pred_tensor shape is ', pred_tensor.shape)
img = 0
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(pred_tensor[img,k])


####  Display for visual check - `gt_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
print('gt_tensor shape is ', gt_tensor.shape)
img = 1
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(gt_tensor[img,k])


####  Display for visual check - `pred_heatmap_scores` is the final result which is passed from  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6, suppress=True)
# print('scatter shape is ', pred_scatt.get_shape())
print('pred_heatmap_scores shape is ', pred_heatmap_scores.shape)
for img in [0,1,2]:
    for k in range(4):
        print('Image ', img , '/ Class ',k,' ------------')
        print(np.min(pred_heatmap_scores[img,k,:,8]))
        print(pred_heatmap_scores[img,k])

####  `GT_HEATMAP_SCORES` 

In [ ]:

print('gt_heatmap_scores shape is ', model_gt_heatmap_scores.shape)
img_id = 0
gt_class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
print('Image : {}  GT ClassIds: {}'.format(img_id, gt_class_ids))

for cls in gt_class_ids:
    print('\nImage ', img_id , '/ Class ',cls,' ------------')
    for box in range(model_gt_heatmap_scores.shape[2]):
        print(model_gt_heatmap_scores[img_id,cls, box])
        if (np.all(model_gt_heatmap_scores[img_id,cls, box, :4] == 0)):
            break

#### `PRED_HEATMAP_SCORES` 

In [ ]:
np_format = {}
float_formatter = lambda x: "%10.4f" % x
int_formatter   = lambda x: "%10d" % x
np_format['float'] = float_formatter
np_format['int']   = int_formatter
np.set_printoptions(linewidth=195, precision=4, floatmode='fixed', threshold =10000, formatter = np_format)
print('pred_heatmap_scores shape is ', model_pr_heatmap_scores.shape)
img_id = 0
pr_class_ids = np.unique(model_pr_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
print('Image : {}  GT ClassIds: {}'.format(img_id, pr_class_ids))

for cls in pr_class_ids:
    print('\nImage ', img_id , '/ Class ',cls,' ------------')
    for box in range(model_pr_heatmap_scores.shape[2]):       
        print(model_pr_heatmap_scores[img_id,cls, box, :8])
        print('{:>86s} {}'.format(' fcn old style scores:  ', model_pr_heatmap_scores[img_id,cls, box,[8,9,10]]))
        print('{:>86s} {}'.format(' fcn alt scores1:  ',model_pr_heatmap_scores[img_id,cls, box,[11,12,13,14,15,16]]))
        print('{:>86s} {}'.format(' fcn_scores2:  '    ,model_pr_heatmap_scores[img_id,cls, box,[17,18,19,20,21,22]]))
        print()        
        if (np.all(model_pr_heatmap_scores[img_id,cls, box, :4] == 0)):
            break

####  `gt_heatmap_scores`  and `fcn_heatmap_scores` 

In [ ]:
np_format = {}
float_formatter = lambda x: "%10.4f" % x
int_formatter   = lambda x: "%10d" % x
np_format['float'] = float_formatter
np_format['int']   = int_formatter
np.set_printoptions(linewidth=195, precision=4, floatmode='fixed', threshold =10000, formatter = np_format)
print('fcn_heatmap_scores shape is ', model_fcn_scores.shape)
img_id = 0
pr_class_ids = np.unique(model_fcn_scores[img_id,:,:,4]).astype(int).tolist()
print('Image : {}  GT ClassIds: {}'.format(img_id, pr_class_ids))

for cls in pr_class_ids:
    print('\nImage ', img_id , '/ Class ',cls,' ------------')
    for box in range(model_fcn_scores.shape[2]):       
        print(model_fcn_scores[img_id,cls, box, :8])
        print('{:>86s} {}'.format(' fcn old style scores:  ', model_fcn_scores[img_id,cls, box,[8,9,10]]))
        print('{:>86s} {}'.format(' fcn alt scores1:  ',model_fcn_scores[img_id,cls, box,[11,12,13,14,15,16]]))
        print('{:>86s} {}'.format(' fcn_scores2:  '    ,model_fcn_scores[img_id,cls, box,[17,18,19,20,21,22]]))
        print()        
        if (np.all(model_fcn_scores[img_id,cls, box, :4] == 0)):
            break

#### `pred_heatmap_norm` 

In [ ]:
print(pred_heatmap_norm.shape)
temp = pred_heatmap_norm
np.set_printoptions(linewidth=150, threshold=10000, suppress=False)
print('  Temp shape :',  temp.shape)
temp_sum = np.sum(temp,axis=(1,2))
print('temp_sum is ', temp_sum.shape)
for i in range(5):
    for j in range(4):
        print('img ',i,' class ', j, ' sum:',temp_sum[i,j],  ' max: ',np.max(temp[i,:,:,j]),' mean: ', np.mean(temp[i,:,:,j]),' min: ', np.min(temp[i,:,:,j]))    

####  `fcn_heatmap`

In [ ]:
np.set_printoptions(linewidth=150, threshold=10000, suppress=False)
temp = fcn_heatmap_norm
print('  Temp shape :',  temp.shape)
temp_sum = np.sum(temp,axis=(1,2))
print('temp_sum is ', temp_sum.shape)
for i in range(5):
    for j in range(4):
        print('img ',i,' class ', j, ' sum:',temp_sum[i,j],  ' max: ',np.max(temp[i,:,:,j]),' mean: ', np.mean(temp[i,:,:,j]),' min: ', np.min(temp[i,:,:,j]))    

In [ ]:
np.set_printoptions(linewidth=150, threshold=10000, suppress=False)
temp = fcn_heatmap
print('  Temp shape :',  temp.shape)
temp_sum = np.sum(temp,axis=(1,2))
print('temp_sum is ', temp_sum.shape)
for i in range(5):
    for j in range(4):
        print('img ',i,' class ', j, ' sum:',temp_sum[i,j],  ' max: ',np.max(temp[i,:,:,j]),' mean: ', np.mean(temp[i,:,:,j]),' min: ', np.min(temp[i,:,:,j]))    


# sess = KB.get_session()
# with sess.as_default():
#     temp = tf.identity(fcn_heatmap
#     np.set_printoptions(linewidth=150, threshold=10000)
#     print('  output shapes :',  temp.get_shape())
#     temp_sum = tf.reduce_sum(temp, [2,3])
#     temp_min = tf.reduce_min(temp, [2,3])
#     temp_max = tf.reduce_max(temp, [2,3])
#     temp_avg = tf.reduce_mean(temp, [2,3])
#     print('temp_sum is ', temp_sum.shape)
#     for i in range(5):
#         for j in range(4):
#                 print('img/cls ',i,'/', j,'  sum:',temp_sum[i,j], 'min',temp_min[i,j] ,'max',temp_max[i,j] ,'avg',temp_avg[i,j])

In [ ]:
np.set_printoptions(linewidth=150, threshold=10000, precision = 6,suppress=False)
sess = KB.get_session()
with sess.as_default():
    tmp = tf.identity(fcn_heatmap)
    print(tf.shape(tmp).eval())
    reduce_max = tf.reduce_max(tmp,axis = [1,2], keepdims=True)
    print(tf.shape(reduce_max).eval())
    reduce_min = tf.reduce_min(tmp, axis = [1,2], keepdims=True)
    print(tf.shape(reduce_min).eval())
    print(reduce_min.eval())
    print(reduce_max.eval())
    y  =  (tmp - reduce_min) / (reduce_max - reduce_min)        
    print(tf.shape(y).eval())

## Display Heatmaps

In [ ]:
from mrcnn.visualize import (plot_one_bbox_heatmap, 
                             plot_3d_heatmap, plot_2d_heatmap, 
                              plot_2d_heatmap_compare, plot_3d_heatmap_compare)
# import matplotlib as plt
# %matplotlib inline
img_id = 0
image_id=outputs['mrcnn_input'][1][img_id,0]
print('Image id: ',image_id, ' Coco ID: ', dataset_train.image_info[image_id]['id'])

coco_class_names = dataset_train.class_names
gt_class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
pr_class_ids = np.unique(model_pr_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
fcn_class_ids = np.unique(model_fcn_scores[img_id,:,:,4]).astype(int).tolist()
print('GT   class ids: ', gt_class_ids)
print('Pred class ids: ', pr_class_ids)
print('FCN  class ids: ', fcn_class_ids)

### 2D Plots

####  2D plot of `model_gt_heatmap` returned from model

In [ ]:
print('Image : {}  ClassIds: {}'.format(img_id, gt_class_ids))
plot_2d_heatmap(model_gt_heatmap, model_gt_heatmap_scores, img_id, gt_class_ids, 
                 class_names = coco_class_names, columns = 3, scale = 4, scaling = 'all')

####  2D plot of `model_fcn_heatmap` returned from model (with Ground Truth Bounding Boxes)

In [ ]:
print('Image : {}  ClassIds: {}'.format(img_id, gt_class_ids))
visualize.plot_2d_heatmap(model_fcn_heatmap, model_gt_heatmap_scores,img_id, gt_class_ids, columns = 3,
                             class_names = coco_class_names, scale = 4, scaling = 'none')

####  2D plot of `model_fcn_heatmap` returned from model (with MRCNN PREDICTED Bounding Boxes)

In [ ]:
print(model_fcn_heatmap.shape)
print('Image : {}  ClassIds: {}'.format(img_id, pr_class_ids))
fig  = visualize.plot_2d_heatmap(model_fcn_heatmap, model_pr_heatmap_scores,img_id, pr_class_ids, columns = 3,
                             class_names = coco_class_names, scale = 4, scaling = 'all')

####  2D plot of `model_fcn_heatmap` returned from model - ALL CLASSES

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    class_ids = np.unique(model_pr_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  Pred ClassIds: {}'.format(img_id, class_ids))
    print('            Gt   ClassIds: {}'.format(np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()))
#     plot_2d_heatmap_with_bboxes(model_fcn_heatmap[...,:class_ids[-1]+1], model_gt_heatmap_scores, img_id, 
#                                 class_ids, class_names = class_names, columns = 2, scale = 4)
    fig = plot_2d_heatmap(model_fcn_heatmap, model_pr_heatmap_scores, img_id,
                                class_names = coco_class_names, columns = 4, scale = 4, scaling = 'all')
#     fig.savefig('fcn_heatmaps_2d.png')

####  2D plot of `pred_heatmap_norm` returned from MRCNN 

In [ ]:
print('Image : {}  ClassIds: {}'.format(img_id, pr_class_ids))
plot_2d_heatmap(model_pr_heatmap, model_pr_heatmap_scores, img_id, pr_class_ids, 
                            class_names = coco_class_names, size = (7,7), scale = 4)

####  2D comparative display `pred_heatmap` / `fcn_heatmap`  

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    class_ids = np.unique(model_pr_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
#     class_ids = list(range(30))
    plot_2d_heatmap_compare(model_pr_heatmap, model_fcn_heatmap, model_pr_heatmap_scores, 
                            img_id, class_ids = class_ids, class_names = coco_class_names, scale = 4, scaling = 'class' )    

### Overlay predictions on image

#### Overlay image with gt_heatmaps

In [ ]:
from mrcnn.visualize import display_heatmaps, display_heatmaps_fcn, display_heatmaps_mrcnn
# visualize.display_image_bw(image)
print(model_gt_heatmap_scores.shape)
class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
print('class ids: ', class_ids)
display_heatmaps_mrcnn(train_batch_x, model_output, 0, hm = 'gt',
                     config = mrcnn_config, class_ids = class_ids, class_names = coco_class_names)

#### Overlay image with pred_heatmaps

In [ ]:
from mrcnn.visualize import display_heatmaps, display_heatmaps_fcn, display_heatmaps_mrcnn
# visualize.display_image_bw(image)
print(model_gt_heatmap_scores.shape)
class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
print('class ids: ', class_ids)
display_heatmaps_mrcnn(train_batch_x, model_output, 0, hm = 'pr',  
                     config = mrcnn_config, class_ids = class_ids, class_names = coco_class_names)

#### Overlay image with fcn_heatmaps

In [ ]:
# visualize.display_heatmaps_mrcnn_fcn(outputs['mrcnn_input'], outputs['mrcnn_output'], 0, heatmap = model_fcn_heatmap, hm = 'pr', columns = 3,
#                      class_ids = gt_class_ids, 
#                                      config = mrcnn_model.config, class_names = coco_class_names, scaling = 'clip')

#### Overlay image with gt_heatmaps and fcn_heatmaps

In [ ]:
visualize.display_heatmaps_compare(outputs['mrcnn_input'], outputs['mrcnn_output'], heatmap = model_fcn_heatmap, image_id = 0, 
                                   hm = 'pr', config = mrcnn_model.config, class_ids = gt_class_ids, class_names = coco_class_names, scaling = 'each')

#### Overlay image with pr_heatmaps and fcn_heatmaps

In [ ]:
visualize.display_heatmaps_compare(outputs['mrcnn_input'], outputs['mrcnn_output'], heatmap = model_fcn_heatmap, image_id = 0, hm = 'pr', 
                     config = mrcnn_model.config, class_ids = pr_class_ids, class_names = coco_class_names, scaling = 'each')

###  3D Plots

####  3D plot of  `gt_heatmap_norm` returned form code above

In [ ]:
for img_id in [0]: ## range(mrcnn_config.BATCH_SIZE):
    print(model_gt_heatmap.shape)
    class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
    plot_3d_heatmap(model_gt_heatmap, img_id, class_ids, class_names = coco_class_names)


####  3D plot of `pred_heatmap_norm` returned form model

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    print(model_pred_heatmap_norm.shape)
    class_ids = np.unique(model_pred_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
    plot_3d_heatmap(model_pred_heatmap_norm, img_id, class_ids, class_names = coco_class_names, zlim = 'all' )


####  3D plot of `fcn_heatmap` returned form model - classes predicted by MRCNN only 

In [ ]:
print(np.max(model_fcn_heatmap), np.min(model_fcn_heatmap))

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    print(model_fcn_heatmap.shape)
    class_ids = np.unique(model_pred_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
    plot_3d_heatmap(model_fcn_heatmap, img_id, class_ids,class_names = coco_class_names, zlim = 'one',scaling = True)


####  3D plot of `fcn_softmax` returned form model - only classes in pred_scores

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    print(model_fcn_heatmap.shape)
    class_ids = np.unique(model_pred_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
    plot_3d_heatmap(model_fcn_softmax, img_id, class_ids,class_names = coco_class_names, scaling = False)

####  3D plot of `fcn_heatmap` returned form model - all classes

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    class_ids = np.unique(model_pred_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
#     plot_3d_heatmap(model_fcn_heatmap, img_id, class_ids,class_names = class_names, size = (8,8), zlim=0)
#     plot_3d_heatmap_all_classes(model_fcn_heatmap[...,:class_ids[-1]+1], img_id, class_names = class_names, size = (8,8), zlim=0.0)
    plot_3d_heatmap(model_fcn_heatmap, img_id, class_names = class_names, columns = 2,scaling = True)

####  3D comparative display `pred_heatmap` / `fcn_heatmap` 

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    print(model_pred_heatmap_norm.shape)
    class_ids = np.unique(model_pred_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
    class_ids.append(1)
    class_ids = list(range(30))
#     print(class_ids)
    plot_3d_heatmap_compare(model_pred_heatmap_norm,model_fcn_heatmap, img_id, class_ids = class_ids, 
                            class_names = class_names, size=(8,8), zlim = 'all' , scaling = True)

####  3D comparative display `gt_heatmap` / `fcn_heatmap` 

In [ ]:
for img_id in [0]: ##range(mrcnn_config.BATCH_SIZE):
    class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
    class_ids.extend([1,2])
    print(class_ids)
    print(type(class_ids))
    class_ids.sort()
    print('Image : {}  ClassIds: {}'.format(img_id, class_ids))
    plot_3d_heatmap_compare(model_gt_heatmap,model_fcn_heatmap, img_id, class_ids, class_names = class_names, size=(8,8), zlim = 0 )

##  FCN Scoring - `FCNScoringLayer`

### Display shapes

In [ ]:
print(model_gt_heatmap.shape)
print(model_gt_heatmap_scores.shape)
print(model_pr_heatmap.shape)
print(model_pr_heatmap_scores.shape)
print(model_fcn_heatmap.shape)
print(model_fcn_softmax.shape)

###  `normalize()`  - `build_fcn_score`

###  `fcn_scoring_graph()`

In [ ]:
def fcn_scoring_graph(input, config):
    in_heatmap, pr_scores = input
    detections_per_image  = pr_scores.shape[2] 
    rois_per_image        = KB.int_shape(pr_scores)[2] 
    img_h, img_w          = config.IMAGE_SHAPE[:2]
    batch_size            = config.BATCH_SIZE
    num_classes           = config.NUM_CLASSES  
    heatmap_scale         = config.HEATMAP_SCALE_FACTOR
    class_column          = 4
    score_column          = 5
    dt_type_column        = 6
    sequence_column       = 7
    norm_score_column     = 8
    print('\n ')
    print('----------------------')
    print('>>> FCN Scoring Layer ')
    print('----------------------')
    logt('in_heatmap.shape  ', in_heatmap)
    logt('pr_hm_scores.shape', pr_scores)
    # rois per image is determined by size of input tensor 
    #   detection mode:   config.TRAIN_ROIS_PER_IMAGE 
    #   ground_truth  :   config.DETECTION_MAX_INSTANCES

    print('    detctions_per_image : ', detections_per_image, 'pr_scores shape', pr_scores.shape )
    print('    rois_per_image      : ', rois_per_image )
    print('    config.DETECTION_MAX_INSTANCES   : ', config.DETECTION_MAX_INSTANCES)
    print('    config.DETECTIONS_PER_CLASS      : ', config.DETECTION_PER_CLASS)

    ##---------------------------------------------------------------------------------------------
    ## Stack non_zero bboxes from PR_SCORES into pt2_dense 
    ##---------------------------------------------------------------------------------------------
    # pt2_ind shape  : [?, 3] : [ {image_index, class_index , roi row_index }]
    # pt2_dense shape: [?, 11] : 
    #    pt2_dense[0:3]  roi coordinates 
    #    pt2_dense[4]    is class id 
    #    pt2_dense[5]    is score from mrcnn    
    #    pt2_dense[6]    is bbox sequence id    
    #    pt2_dense[7]    is normalized score (per class)    
    #-----------------------------------------------------------------------------
    pt2_sum = tf.reduce_sum(tf.abs(pr_scores[:,:,:,:4]), axis=-1)
    pt2_ind = tf.where(pt2_sum > 0)
    pt2_dense = tf.gather_nd(pr_scores, pt2_ind)
    logt('in_heatmap       ', in_heatmap)
    logt('pr_scores.shape  ', pr_scores)
    logt('pt2_sum shape    ', pt2_sum)
    logt('pt2_ind shape    ', pt2_ind)
    logt('pt2_dense shape  ', pt2_dense)


    ##---------------------------------------------------------------------------------------------
    ##  Build mean and convariance tensors for bounding boxes
    ##---------------------------------------------------------------------------------------------
    # bboxes_scaled = tf.to_int32(tf.round(pt2_dense[...,0:4])) / heatmap_scale
    bboxes_scaled = pt2_dense[...,0:4] / heatmap_scale
    width  = bboxes_scaled[:,3] - bboxes_scaled[:,1]      # x2 - x1
    height = bboxes_scaled[:,2] - bboxes_scaled[:,0]
    cx     = bboxes_scaled[:,1] + ( width  / 2.0)
    cy     = bboxes_scaled[:,0] + ( height / 2.0)
    # means  = tf.stack((cx,cy),axis = -1)
    covar  = tf.stack((width * 0.5 , height * 0.5), axis = -1)
    covar  = tf.sqrt(covar)            

    
    ##---------------------------------------------------------------------------------------------
    ##  build indices and extract heatmaps corresponding to each bounding boxes' class id
    ##---------------------------------------------------------------------------------------------
    hm_indices = tf.cast(pt2_ind[:, :2],dtype=tf.int32)
    logt('hm_indices  ',  hm_indices)
    pt2_heatmaps = tf.transpose(in_heatmap, [0,3,1,2])
    logt('pt2_heatmaps',  pt2_heatmaps)
    pt2_heatmaps = tf.gather_nd(pt2_heatmaps, hm_indices )
    logt('pt2_heatmaps',  pt2_heatmaps)

    ##--------------------------------------------------------------------------------------------
    ## (0) Generate scores using prob_grid and pt2_dense - (NEW METHOD added 09-21-2018)
    ##--------------------------------------------------------------------------------------------
    old_style_scores = tf.map_fn(build_hm_score_v2, [pt2_heatmaps, bboxes_scaled, pt2_dense[:, norm_score_column]], 
                                 dtype = tf.float32, swap_memory = True)
    logt('old_style_scores',  old_style_scores)                                 
                                                                      
    # old_style_scores = tf.scatter_nd(pt2_ind, old_style_scores, 
                                     # [batch_size, num_classes, rois_per_image, KB.int_shape(old_style_scores)[-1]],
                                     # name = 'scores_scattered')
    # print('    old_style_scores        :',  old_style_scores.get_shape(), KB.int_shape(old_style_scores))                                 
                                     
    ##---------------------------------------------------------------------------------------------
    ## generate score based on gaussian using bounding box masks 
    ##---------------------------------------------------------------------------------------------
    alt_scores_1 = tf.map_fn(build_hm_score_v3, [pt2_heatmaps, cy, cx,covar], dtype=tf.float32)    
    logt('alt_scores_1 ', alt_scores_1 )

    ##---------------------------------------------------------------------------------------------
    ##  Scatter back to per-class tensor /  normalize by class
    ##---------------------------------------------------------------------------------------------
    alt_scores_1_norm = tf.scatter_nd(pt2_ind, alt_scores_1, 
                                    [batch_size, num_classes, detections_per_image, KB.int_shape(alt_scores_1)[-1]],
                                    name='alt_scores_1_norm')
    logt('alt_scores_1_scattered', alt_scores_1_norm )  
    alt_scores_1_norm = normalize_scores(alt_scores_1_norm)
    logt('alt_scores_1_norm(by_class)', alt_scores_1_norm)
    alt_scores_1_norm = tf.gather_nd(alt_scores_1_norm, pt2_ind)
    logt('alt_scores_1_norm(by_image)', alt_scores_1_norm)

###################################################################################################################
    ## Note: Running this scoring method yields the exact same final result as alt_score_1, and is therefore redundant
    ##-------------------------------------------------------------------------------------
    ## Normalize input heatmap normalization (per class) to calculate alt_score_2
    ##-------------------------------------------------------------------------------------
    print('\n    Normalize heatmap within each class !-------------------------------------')         
    in_heatmap_norm = tf.transpose(in_heatmap, [0,3,1,2])

    print('    in_heatmap_norm : ', in_heatmap_norm.get_shape(), 'Keras tensor ', KB.is_keras_tensor(in_heatmap_norm) )      
    ## normalize in class
    normalizer = tf.reduce_max(in_heatmap_norm, axis=[-2,-1], keepdims = True)
    normalizer = tf.where(normalizer < 1.0e-15,  tf.ones_like(normalizer), normalizer)
    in_heatmap_norm = in_heatmap_norm / normalizer
    # gauss_heatmap_sum_normalized = gauss_heatmap_sum / normalizer
    print('    normalizer shape   : ', normalizer.shape)   
    print('    normalized heatmap : ', in_heatmap_norm.shape   ,' Keras tensor ', KB.is_keras_tensor(in_heatmap_norm) )

    ##---------------------------------------------------------------------------------------------
    ##  build indices and extract heatmaps corresponding to each bounding boxes' class id
    ##  build alternative scores#  based on normalized/sclaked clipped heatmap
    ##---------------------------------------------------------------------------------------------
    hm_indices = tf.cast(pt2_ind[:, :2],dtype=tf.int32)
    logt('hm_indices shape',  hm_indices)
    
    pt2_heatmaps = tf.gather_nd(in_heatmap_norm, hm_indices )
    logt('pt2_heatmaps',  pt2_heatmaps)

    alt_scores_2 = tf.map_fn(build_hm_score_v3, [pt2_heatmaps, cy, cx,covar], dtype=tf.float32)    
    logt('alt_scores_2',alt_scores_2)
    
    alt_scores_2_norm = tf.scatter_nd(pt2_ind, alt_scores_2, 
                                     [batch_size, num_classes, rois_per_image, KB.int_shape(alt_scores_2)[-1]], name = 'alt_scores_2')  
    logt('alt_scores_2(scattered)', alt_scores_2_norm ) 
    
    alt_scores_2_norm = normalize_scores(alt_scores_2_norm)
    logt('alt_scores_2_norm(by_class)', alt_scores_2_norm)
    
    alt_scores_2_norm = tf.gather_nd(alt_scores_2_norm, pt2_ind)
    logt('alt_scores_2_norm(by_image)', alt_scores_2_norm)
####################################################################################################################
    
    ##--------------------------------------------------------------------------------------------
    ##  Append alt_scores_1, alt_scores_1_norm to yield fcn_scores_dense 
    ##--------------------------------------------------------------------------------------------
    fcn_scores_dense = tf.concat([pt2_dense[:, : norm_score_column+1], old_style_scores, alt_scores_1, alt_scores_1_norm, alt_scores_2, alt_scores_2_norm], 
                                  axis = -1, name = 'fcn_scores_dense')
    logt('fcn_scores_dense    ', fcn_scores_dense )  

    ##---------------------------------------------------------------------------------------------
    ##  Scatter back to per-image tensor 
    ##---------------------------------------------------------------------------------------------
    seq_ids = tf.to_int32( rois_per_image - pt2_dense[:, sequence_column] )
    scatter_ind= tf.stack([hm_indices[:,0], seq_ids], axis = -1, name = 'scatter_ind')
    fcn_scores_by_class = tf.scatter_nd(pt2_ind, fcn_scores_dense, 
                                        [batch_size, num_classes, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_hm_scores')
    # fcn_scores_by_image = tf.scatter_nd(scatter_ind, fcn_scores_dense, 
                                        # [batch_size, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_hm_scores_by_image')
    logt('seq_ids             ', seq_ids) 
    logt('sscatter_ids        ', scatter_ind)
    logt('fcn_scores_by_class ', fcn_scores_by_class) 
    # logt('fcn_scores_by_image ', fcn_scores_by_image) 
    logt('complete')    
   
    return fcn_scores_by_class

###  `fcn_scoreing_graph()` previous version

In [ ]:
# sess = KB.get_session()
# with sess.as_default():
##----------------------------------------------------------------------------------------------------------------------          
##   build_fcn_scores 
##   in_heatmap: float32
##   pr_scores : float32
##----------------------------------------------------------------------------------------------------------------------          
# def fcn_scoring_graph(input, config):
# in_heatmap, pr_scores = input
detections_per_image  = pr_scores.shape[2] 
rois_per_image  = KB.int_shape(pr_scores)[2] 
img_h, img_w    = config.IMAGE_SHAPE[:2]
batch_size      = config.BATCH_SIZE
num_classes     = config.NUM_CLASSES  
heatmap_scale   = config.HEATMAP_SCALE_FACTOR
print('\n ')
print('----------------------')
print('>>> FCN Scoring Layer ')
print('----------------------')
print('    in_heatmap.shape     :', in_heatmap.shape  , KB.int_shape(in_heatmap)  , 'Keras tensor ', KB.is_keras_tensor(in_heatmap))
print('    pr_hm_scores.shape   :', pr_scores.shape, KB.int_shape(pr_scores), 'Keras tensor ', KB.is_keras_tensor(pr_scores))
# rois per image is determined by size of input tensor 
#   detection mode:   config.TRAIN_ROIS_PER_IMAGE 
#   ground_truth  :   config.DETECTION_MAX_INSTANCES

print('    detctions_per_image : ', detections_per_image, 'pr_scores shape', pr_scores.shape )
print('    rois_per_image      : ', rois_per_image )
print('    config.DETECTION_MAX_INSTANCES   : ', config.DETECTION_MAX_INSTANCES)
print('    config.DETECTIONS_PER_CLASS      : ', config.DETECTION_PER_CLASS)



##---------------------------------------------------------------------------------------------
## Stack non_zero bboxes from PR_SCORES into pt2_dense 
##---------------------------------------------------------------------------------------------
# pt2_ind shape  : [?, 3] : [ {image_index, class_index , roi row_index }]
# pt2_dense shape: [?, 11] : 
#    pt2_dense[0:3]  roi coordinates 
#    pt2_dense[4]    is class id 
#    pt2_dense[5]    is score from mrcnn    
#    pt2_dense[6]    is bbox sequence id    
#    pt2_dense[7]    is normalized score (per class)    
#-----------------------------------------------------------------------------
pt2_sum = tf.reduce_sum(tf.abs(pr_scores[:,:,:,:4]), axis=-1)
pt2_ind = tf.where(pt2_sum > 0)
pt2_dense = tf.gather_nd(pr_scores, pt2_ind)
bboxes = tf.to_int32(tf.round(pt2_dense[...,0:4]))
print('    in_heatmap          : ', in_heatmap.shape, KB.int_shape(in_heatmap))
print('    pr_scores.shape     : ', pr_scores.shape, KB.int_shape(pr_scores))
print('    pt2_sum shape       : ', pt2_sum.shape, KB.int_shape(pt2_sum))
print('    pt2_ind shape       : ', pt2_ind.shape, KB.int_shape(pt2_ind))     
#     print(pt2_ind.eval())
print('    pt2_dense shape     : ', pt2_dense.get_shape(), KB.int_shape(pt2_dense))
print('    bboxes  shape       : ', KB.int_shape(bboxes))
#     print('    pt2_dense        : ', pt2_dense.eval())   


##---------------------------------------------------------------------------------------------
##  Build mean and convariance tensors for Multivariate Normal Distribution 
##---------------------------------------------------------------------------------------------
bboxes = bboxes / heatmap_scale
width  = bboxes[:,3] - bboxes[:,1]      # x2 - x1
height = bboxes[:,2] - bboxes[:,0]
cx     = bboxes[:,1] + ( width  / 2.0)
cy     = bboxes[:,0] + ( height / 2.0)
# means  = tf.stack((cx,cy),axis = -1)
covar  = tf.stack((width * 0.5 , height * 0.5), axis = -1)
covar  = tf.sqrt(covar)            

##---------------------------------------------------------------------------------------------
##  build indices and extract heatmaps corresponding to each bounding boxes' class id
##---------------------------------------------------------------------------------------------
hm_indices = tf.cast(pt2_ind[:, :2],dtype=tf.int32)
print('    hm_indices shape         :',  hm_indices.get_shape(), KB.int_shape(hm_indices))
pt2_heatmaps = tf.transpose(in_heatmap, [0,3,1,2])
print('    pt2_heatmaps             :',  pt2_heatmaps.get_shape(), KB.int_shape(pt2_heatmaps))
pt2_heatmaps = tf.gather_nd(pt2_heatmaps, hm_indices )
print('    pt2_heatmaps after gather:',  pt2_heatmaps.get_shape(), KB.int_shape(pt2_heatmaps))
 

##---------------------------------------------------------------------------------------------
## generate score based on gaussian using bounding box masks 
##---------------------------------------------------------------------------------------------
map_output = tf.map_fn(build_fcn_score, [pt2_heatmaps, cy, cx,covar], dtype=tf.float32)    
scores = map_output[:,0]
print(type(map_output))
print('    map_output                 : ', map_output.shape ,' Keras tensor ', KB.is_keras_tensor(map_output) )  

# scores = tf.map_fn(build_fcn_score, [pt2_heatmaps, cy, cx,covar], dtype=tf.float32)    
scores = tf.expand_dims(scores, axis = -1)
print('    scores                  : ', scores.shape ,' Keras tensor ', KB.is_keras_tensor(scores) )  
# print('    mask_sum                : ', mask_sum.shape ,' Keras tensor ', KB.is_keras_tensor(mask_sum) )  
# print('    mask_area               : ', mask_area.shape ,' Keras tensor ', KB.is_keras_tensor(mask_area) )  
print('heatmap, pr_scores, cx,cy,cvoar', in_heatmap.dtype, scores.dtype, bboxes.dtype, cx.dtype, cy.dtype, covar.dtype)


##---------------------------------------------------------------------------------------------
##  Scatter back to per-class tensor 
##---------------------------------------------------------------------------------------------
scores_by_class = tf.scatter_nd(pt2_ind, scores, [batch_size, num_classes, detections_per_image, 1], name='scores_by_class')
print('    scores_by_class        : ', scores_by_class.shape ,' Keras tensor ', KB.is_keras_tensor(scores_by_class) )  
norm_scores = normalize(scores_by_class)
print('    norm_score_by_class    : ', norm_scores.shape, KB.int_shape(norm_scores))
norm_scores = tf.gather_nd(norm_scores, pt2_ind)
print('    norm_score_dense       : ', norm_scores.shape, KB.int_shape(norm_scores))

##--------------------------------------------------------------------------------------------
##  Append scores, norm_scores to yield fcn_scores_dense 
##--------------------------------------------------------------------------------------------
fcn_scores_dense = tf.concat([pt2_dense , scores, norm_scores, map_output[:,1:]], axis = -1, name = 'fcn_scores_dense')

print('    fcn_scores_dense    : ', fcn_scores_dense.shape, KB.int_shape(fcn_scores_dense) ,
            ' Keras tensor ', KB.is_keras_tensor(fcn_scores_dense) )  
 
##---------------------------------------------------------------------------------------------
##  Scatter back to per-image tensor 
##---------------------------------------------------------------------------------------------
seq_ids = tf.to_int32( rois_per_image - pt2_dense[:,6] )
scatter_ind= tf.stack([hm_indices[:,0], seq_ids], axis = -1, name = 'scatter_ind')
fcn_scores_by_class = tf.scatter_nd(pt2_ind, fcn_scores_dense, 
                                    [batch_size, num_classes, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_scores')
fcn_scores_by_image = tf.scatter_nd(scatter_ind, fcn_scores_dense, 
                                    [batch_size, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_scores_by_image')
print('    seq_ids                : ', KB.int_shape(seq_ids) ,' Keras tensor ', KB.is_keras_tensor(seq_ids) )  
print('    sscatter_ids           : ', KB.int_shape(scatter_ind),' Keras tensor ', KB.is_keras_tensor(scatter_ind) )  
print('    fcn_scores_by_class    : ', KB.int_shape(fcn_scores_by_class) ,' Keras tensor ', KB.is_keras_tensor(fcn_scores_by_class) )  
print('    fcn_scores_by_image    : ', KB.int_shape(fcn_scores_by_image) ,' Keras tensor ', KB.is_keras_tensor(fcn_scores_by_image) )  
print('    complete')    
   
#     return fcn_scores_by_class


# pt2_heatmaps = tf.transpose(pt2_heatmaps, [2,0,1])
# pt2_heatmaps = tf.transpose(pt2_heatmaps, [0,3,1,2])
# print('    selected heatmaps shape  :',  pt2_heatmaps.get_shape(), KB.int_shape(pt2_heatmaps))
# pt2_hm_reshape = tf.reshape(pt2_heatmaps,[-1, pt2_heatmaps.shape[2], pt2_heatmaps.shape[3]])

###  `fcn_scoreing_graph()`  Nov 30

In [ ]:
from mrcnn.chm_layer import build_hm_score_v2, build_hm_score_v3, normalize_scores
from mrcnn.utils import logt
##-------------------------------------------------------------------------------------------------------
##   score fcn heatmaps : gen scores from heatmap
##-------------------------------------------------------------------------------------------------------
##   We use the coordinates of the bounding boxes passed in pr_scores to calculate 
##   the score of bounding boxes overlaid on the heatmap produced by the fcn_layer
##   - convert the pr_scores (or gt_hm_scores) from a per_class/per_bbox tensor to a per_class tensor
##     [BATCH_SIZE, NUM_CLASSES, DETECTIONS_PER_CLASS, 11] --> [BATCH_SIZE, DETECTIONS_MAX_INSTANCES, 11]
##   - Extract non-zero bounding boxes
##   - calculate the Cy, Cx, and Covar of the bounding boxes 
##   - Clip the heatmap by using masks centered on Cy,Cx and +/- Covar_Y, Covar_X
##-------------------------------------------------------------------------------------------------------
def fcn_scoring_graph(input, config):
    in_heatmap, pr_scores = input
    detections_per_image  = pr_scores.shape[2] 
    rois_per_image  = KB.int_shape(pr_scores)[2] 
    img_h, img_w    = config.IMAGE_SHAPE[:2]
    batch_size      = config.BATCH_SIZE
    num_classes     = config.NUM_CLASSES  
    heatmap_scale   = config.HEATMAP_SCALE_FACTOR
    print('\n ')
    print('----------------------')
    print('>>> FCN Scoring Layer ')
    print('----------------------')
    print('    in_heatmap.shape     :', in_heatmap.shape  , KB.int_shape(in_heatmap)  , 'Keras tensor ', KB.is_keras_tensor(in_heatmap))
    print('    pr_hm_scores.shape   :', pr_scores.shape, KB.int_shape(pr_scores), 'Keras tensor ', KB.is_keras_tensor(pr_scores))
    # rois per image is determined by size of input tensor 
    #   detection mode:   config.TRAIN_ROIS_PER_IMAGE 
    #   ground_truth  :   config.DETECTION_MAX_INSTANCES

    print('    detctions_per_image : ', detections_per_image, 'pr_scores shape', pr_scores.shape )
    print('    rois_per_image      : ', rois_per_image )
    print('    config.DETECTION_MAX_INSTANCES   : ', config.DETECTION_MAX_INSTANCES)
    print('    config.DETECTIONS_PER_CLASS      : ', config.DETECTION_PER_CLASS)

    ##---------------------------------------------------------------------------------------------
    ## Stack non_zero bboxes from PR_SCORES into pt2_dense 
    ##---------------------------------------------------------------------------------------------
    # pt2_ind shape  : [?, 3] : [ {image_index, class_index , roi row_index }]
    # pt2_dense shape: [?, 11] : 
    #    pt2_dense[0:3]  roi coordinates 
    #    pt2_dense[4]    is class id 
    #    pt2_dense[5]    is score from mrcnn    
    #    pt2_dense[6]    is bbox sequence id    
    #    pt2_dense[7]    is normalized score (per class)    
    #-----------------------------------------------------------------------------
    pt2_sum = tf.reduce_sum(tf.abs(pr_scores[:,:,:,:4]), axis=-1)
    pt2_ind = tf.where(pt2_sum > 0)
    pt2_dense = tf.gather_nd(pr_scores, pt2_ind)
    print('    in_heatmap          : ', in_heatmap.shape, KB.int_shape(in_heatmap))
    print('    pr_scores.shape     : ', pr_scores.shape, KB.int_shape(pr_scores))
    print('    pt2_sum shape       : ', pt2_sum.shape, KB.int_shape(pt2_sum))
    print('    pt2_ind shape       : ', pt2_ind.shape, KB.int_shape(pt2_ind))     
    print('    pt2_dense shape     : ', pt2_dense.get_shape(), KB.int_shape(pt2_dense))
    # print('    bboxes  shape       : ', KB.int_shape(bboxes))


    ##---------------------------------------------------------------------------------------------
    ##  Build mean and convariance tensors for bounding boxes
    ##---------------------------------------------------------------------------------------------
    # bboxes_scaled = tf.to_int32(tf.round(pt2_dense[...,0:4])) / heatmap_scale
    bboxes_scaled = pt2_dense[...,0:4] / heatmap_scale
    width  = bboxes_scaled[:,3] - bboxes_scaled[:,1]      # x2 - x1
    height = bboxes_scaled[:,2] - bboxes_scaled[:,0]
    cx     = bboxes_scaled[:,1] + ( width  / 2.0)
    cy     = bboxes_scaled[:,0] + ( height / 2.0)
    # means  = tf.stack((cx,cy),axis = -1)
    covar  = tf.stack((width * 0.5 , height * 0.5), axis = -1)
    covar  = tf.sqrt(covar)            

    
    ##---------------------------------------------------------------------------------------------
    ##  build indices and extract heatmaps corresponding to each bounding boxes' class id
    ##---------------------------------------------------------------------------------------------
    hm_indices = tf.cast(pt2_ind[:, :2],dtype=tf.int32)
    print('    hm_indices shape         :',  hm_indices.get_shape(), KB.int_shape(hm_indices))
    pt2_heatmaps = tf.transpose(in_heatmap, [0,3,1,2])
    print('    pt2_heatmaps             :',  pt2_heatmaps.get_shape(), KB.int_shape(pt2_heatmaps))
    pt2_heatmaps = tf.gather_nd(pt2_heatmaps, hm_indices )
    print('    pt2_heatmaps after gather:',  pt2_heatmaps.get_shape(), KB.int_shape(pt2_heatmaps))

    ##--------------------------------------------------------------------------------------------
    ## (0) Generate scores using prob_grid and pt2_dense - (NEW METHOD added 09-21-2018)
    ##--------------------------------------------------------------------------------------------
    old_style_scores = tf.map_fn(build_hm_score_v2, [pt2_heatmaps, bboxes_scaled, pt2_dense[:,7]], 
                                 dtype = tf.float32, swap_memory = True)
    print('    old_style_scores        :',  old_style_scores.get_shape(), KB.int_shape(old_style_scores))                                 
                                                                      
    # old_style_scores = tf.scatter_nd(pt2_ind, old_style_scores, 
                                     # [batch_size, num_classes, rois_per_image, KB.int_shape(old_style_scores)[-1]],
                                     # name = 'scores_scattered')
    # print('    old_style_scores        :',  old_style_scores.get_shape(), KB.int_shape(old_style_scores))                                 
                                     
    ##---------------------------------------------------------------------------------------------
    ## generate score based on gaussian using bounding box masks 
    ##---------------------------------------------------------------------------------------------
    alt_scores_1 = tf.map_fn(build_hm_score_v3, [pt2_heatmaps, cy, cx,covar], dtype=tf.float32)    
    # alt_scores_1 = tf.expand_dims(alt_scores_1, axis = -1)
    print('    alt_scores_1                  : ', alt_scores_1.shape ,' Keras tensor ', KB.is_keras_tensor(alt_scores_1) )  

    ##---------------------------------------------------------------------------------------------
    ##  Scatter back to per-class tensor /  normalize by class
    ##---------------------------------------------------------------------------------------------
    alt_scores_1_norm = tf.scatter_nd(pt2_ind, alt_scores_1, 
                                    [batch_size, num_classes, detections_per_image, KB.int_shape(alt_scores_1)[-1]],
                                    name='alt_scores_1_norm')
    print('    alt_scores_1_scattered            : ', alt_scores_1_norm.shape ,' Keras tensor ', KB.is_keras_tensor(alt_scores_1_norm) )  
    alt_scores_1_norm = normalize_scores(alt_scores_1_norm)
    print('    alt_scores_1_norm(by_class)  : ', alt_scores_1_norm.shape, KB.int_shape(alt_scores_1_norm))
    alt_scores_1_norm = tf.gather_nd(alt_scores_1_norm, pt2_ind)
    print('    alt_scores_1_norm(by_image)  : ', alt_scores_1_norm.shape, KB.int_shape(alt_scores_1_norm))

###################################################################################################################
    ## Note: Running this scoring method yields the exact same final result as alt_score_1, and is therefore redundant
    ##-------------------------------------------------------------------------------------
    ## Normalize input heatmap normalization (per class) to calculate alt_score_2
    ##-------------------------------------------------------------------------------------
    print('\n    Normalize heatmap within each class !-------------------------------------')         
    in_heatmap_norm = tf.transpose(in_heatmap, [0,3,1,2])

    print('    in_heatmap_norm : ', in_heatmap_norm.get_shape(), 'Keras tensor ', KB.is_keras_tensor(in_heatmap_norm) )      
    ## normalize in class
    normalizer = tf.reduce_max(in_heatmap_norm, axis=[-2,-1], keepdims = True)
    normalizer = tf.where(normalizer < 1.0e-15,  tf.ones_like(normalizer), normalizer)
    in_heatmap_norm = in_heatmap_norm / normalizer
    # gauss_heatmap_sum_normalized = gauss_heatmap_sum / normalizer
    print('    normalizer shape   : ', normalizer.shape)   
    print('    normalized heatmap : ', in_heatmap_norm.shape   ,' Keras tensor ', KB.is_keras_tensor(in_heatmap_norm) )

    ##---------------------------------------------------------------------------------------------
    ##  build indices and extract heatmaps corresponding to each bounding boxes' class id
    ##  build alternative scores#  based on normalized/sclaked clipped heatmap
    ##---------------------------------------------------------------------------------------------
    hm_indices = tf.cast(pt2_ind[:, :2],dtype=tf.int32)
    logt('hm_indices shape',  hm_indices)
    
    pt2_heatmaps = tf.gather_nd(in_heatmap_norm, hm_indices )
    logt('pt2_heatmaps',  pt2_heatmaps)

    alt_scores_2 = tf.map_fn(build_hm_score_v3, [pt2_heatmaps, cy, cx,covar], dtype=tf.float32)    
    logt('alt_scores_2',alt_scores_2)
    
    alt_scores_2_norm = tf.scatter_nd(pt2_ind, alt_scores_2, 
                                     [batch_size, num_classes, rois_per_image, KB.int_shape(alt_scores_2)[-1]], name = 'alt_scores_2')  
    logt('alt_scores_2(scattered)', alt_scores_2_norm ) 
    
    alt_scores_2_norm = normalize_scores(alt_scores_2_norm)
    logt('alt_scores_2_norm(by_class)', alt_scores_2_norm)
    
    alt_scores_2_norm = tf.gather_nd(alt_scores_2_norm, pt2_ind)
    logt('alt_scores_2_norm(by_image)', alt_scores_2_norm)
####################################################################################################################
    
    ##--------------------------------------------------------------------------------------------
    ##  Append alt_scores_1, alt_scores_1_norm to yield fcn_scores_dense 
    ##--------------------------------------------------------------------------------------------
    fcn_scores_dense = tf.concat([pt2_dense[:,:8], old_style_scores, alt_scores_1, alt_scores_1_norm, alt_scores_2, alt_scores_2_norm], 
                                  axis = -1, name = 'fcn_scores_dense')
    logt('fcn_scores_dense    ', fcn_scores_dense )  

    ##---------------------------------------------------------------------------------------------
    ##  Scatter back to per-image tensor 
    ##---------------------------------------------------------------------------------------------
    seq_ids = tf.to_int32( rois_per_image - pt2_dense[:,6] )
    scatter_ind= tf.stack([hm_indices[:,0], seq_ids], axis = -1, name = 'scatter_ind')
    fcn_scores_by_class = tf.scatter_nd(pt2_ind, fcn_scores_dense, 
                                        [batch_size, num_classes, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_scores')
    fcn_scores_by_image = tf.scatter_nd(scatter_ind, fcn_scores_dense, 
                                        [batch_size, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_scores')
    logt('seq_ids             ', seq_ids) 
    logt('sscatter_ids        ', scatter_ind)
    logt('fcn_scores_by_class ', fcn_scores_by_class) 
    logt('fcn_scores_by_image ', fcn_scores_by_image) 
    logt('complete')    
   
    return fcn_scores_by_class
    

###  `fcn_scoreing_graph()`  Nov 29

In [ ]:
from mrcnn.chm_layer import build_hm_score_v2, build_hm_score_v3, normalize_scores
def fcn_scoring_graph(input, config):
    in_heatmap, pr_scores = input
    detections_per_image  = pr_scores.shape[2] 
    rois_per_image  = KB.int_shape(pr_scores)[2] 
    img_h, img_w    = config.IMAGE_SHAPE[:2]
    batch_size      = config.BATCH_SIZE
    num_classes     = config.NUM_CLASSES  
    heatmap_scale   = config.HEATMAP_SCALE_FACTOR
    print('\n ')
    print('----------------------')
    print('>>> FCN Scoring Layer ')
    print('----------------------')
    print('    in_heatmap.shape     :', in_heatmap.shape  , KB.int_shape(in_heatmap)  , 'Keras tensor ', KB.is_keras_tensor(in_heatmap))
    print('    pr_hm_scores.shape   :', pr_scores.shape, KB.int_shape(pr_scores), 'Keras tensor ', KB.is_keras_tensor(pr_scores))
    # rois per image is determined by size of input tensor 
    #   detection mode:   config.TRAIN_ROIS_PER_IMAGE 
    #   ground_truth  :   config.DETECTION_MAX_INSTANCES

    print('    detctions_per_image : ', detections_per_image, 'pr_scores shape', pr_scores.shape )
    print('    rois_per_image      : ', rois_per_image )
    print('    config.DETECTION_MAX_INSTANCES   : ', config.DETECTION_MAX_INSTANCES)
    print('    config.DETECTIONS_PER_CLASS      : ', config.DETECTION_PER_CLASS)

    ##---------------------------------------------------------------------------------------------
    ## Stack non_zero bboxes from PR_SCORES into pt2_dense 
    ##---------------------------------------------------------------------------------------------
    # pt2_ind shape  : [?, 3] : [ {image_index, class_index , roi row_index }]
    # pt2_dense shape: [?, 11] : 
    #    pt2_dense[0:3]  roi coordinates 
    #    pt2_dense[4]    is class id 
    #    pt2_dense[5]    is score from mrcnn    
    #    pt2_dense[6]    is bbox sequence id    
    #    pt2_dense[7]    is normalized score (per class)    
    #-----------------------------------------------------------------------------
    pt2_sum = tf.reduce_sum(tf.abs(pr_scores[:,:,:,:4]), axis=-1)
    pt2_ind = tf.where(pt2_sum > 0)
    pt2_dense = tf.gather_nd(pr_scores, pt2_ind)
    print('    in_heatmap          : ', in_heatmap.shape, KB.int_shape(in_heatmap))
    print('    pr_scores.shape     : ', pr_scores.shape, KB.int_shape(pr_scores))
    print('    pt2_sum shape       : ', pt2_sum.shape, KB.int_shape(pt2_sum))
    print('    pt2_ind shape       : ', pt2_ind.shape, KB.int_shape(pt2_ind))     
    print('    pt2_dense shape     : ', pt2_dense.get_shape(), KB.int_shape(pt2_dense))
    # print('    bboxes  shape       : ', KB.int_shape(bboxes))


    ##---------------------------------------------------------------------------------------------
    ##  Build mean and convariance tensors for bounding boxes
    ##---------------------------------------------------------------------------------------------
    # bboxes_scaled = tf.to_int32(tf.round(pt2_dense[...,0:4])) / heatmap_scale
    bboxes_scaled = pt2_dense[...,0:4] / heatmap_scale
    width  = bboxes_scaled[:,3] - bboxes_scaled[:,1]      # x2 - x1
    height = bboxes_scaled[:,2] - bboxes_scaled[:,0]
    cx     = bboxes_scaled[:,1] + ( width  / 2.0)
    cy     = bboxes_scaled[:,0] + ( height / 2.0)
    # means  = tf.stack((cx,cy),axis = -1)
    covar  = tf.stack((width * 0.5 , height * 0.5), axis = -1)
    covar  = tf.sqrt(covar)            


    ##---------------------------------------------------------------------------------------------
    ##  build indices and extract heatmaps corresponding to each bounding boxes' class id
    ##---------------------------------------------------------------------------------------------
    hm_indices = tf.cast(pt2_ind[:, :2],dtype=tf.int32)
    print('    hm_indices shape         :',  hm_indices.get_shape(), KB.int_shape(hm_indices))
    pt2_heatmaps = tf.transpose(in_heatmap, [0,3,1,2])
    print('    pt2_heatmaps             :',  pt2_heatmaps.get_shape(), KB.int_shape(pt2_heatmaps))
    pt2_heatmaps = tf.gather_nd(pt2_heatmaps, hm_indices )
    print('    pt2_heatmaps after gather:',  pt2_heatmaps.get_shape(), KB.int_shape(pt2_heatmaps))
    print('    bboxes scaled            :',  bboxes_scaled.get_shape(), KB.int_shape(bboxes_scaled))

    ##--------------------------------------------------------------------------------------------
    ## (0) Generate scores using prob_grid and pt2_dense - (NEW METHOD added 09-21-2018)
    ##--------------------------------------------------------------------------------------------
    old_style_scores = tf.map_fn(build_hm_score_v2, [pt2_heatmaps, bboxes_scaled, pt2_dense[:,7]], 
                                 dtype = tf.float32, swap_memory = True)
    print('    old_style_scores        :',  old_style_scores.get_shape(), KB.int_shape(old_style_scores))                                 
                                                                      
    # old_style_scores = tf.scatter_nd(pt2_ind, old_style_scores, 
                                     # [batch_size, num_classes, rois_per_image, KB.int_shape(old_style_scores)[-1]],
                                     # name = 'scores_scattered')
    # print('    old_style_scores        :',  old_style_scores.get_shape(), KB.int_shape(old_style_scores))                                 
                                     
    ##---------------------------------------------------------------------------------------------
    ## generate score based on gaussian using bounding box masks 
    ##---------------------------------------------------------------------------------------------
    scores = tf.map_fn(build_hm_score_v3, [pt2_heatmaps, cy, cx,covar], dtype=tf.float32)    
    # scores = tf.expand_dims(scores, axis = -1)
    print('    scores                  : ', scores.shape ,' Keras tensor ', KB.is_keras_tensor(scores) )  

    ##---------------------------------------------------------------------------------------------
    ##  Scatter back to per-class tensor 
    ##---------------------------------------------------------------------------------------------
    scores_by_class = tf.scatter_nd(pt2_ind, scores, 
                                    [batch_size, num_classes, detections_per_image, KB.int_shape(scores)[-1]],
                                    name='scores_by_class')
    print('    scores_by_class        : ', scores_by_class.shape ,' Keras tensor ', KB.is_keras_tensor(scores_by_class) )  
    norm_scores = normalize_scores(scores_by_class)
    print('    norm_score_by_class    : ', norm_scores.shape, KB.int_shape(norm_scores))
    norm_scores = tf.gather_nd(norm_scores, pt2_ind)
    print('    norm_score_(by_image)  : ', norm_scores.shape, KB.int_shape(norm_scores))

    ##--------------------------------------------------------------------------------------------
    ##  Append scores, norm_scores to yield fcn_scores_dense 
    ##--------------------------------------------------------------------------------------------
    fcn_scores_dense = tf.concat([pt2_dense[:,:8], old_style_scores, scores, norm_scores], axis = -1, name = 'fcn_scores_dense')
    print('    fcn_scores_dense    : ', fcn_scores_dense.shape, KB.int_shape(fcn_scores_dense) ,
                ' Keras tensor ', KB.is_keras_tensor(fcn_scores_dense) )  

    ##---------------------------------------------------------------------------------------------
    ##  Scatter back to per-image tensor 
    ##---------------------------------------------------------------------------------------------
    seq_ids = tf.to_int32( rois_per_image - pt2_dense[:,6] )
    scatter_ind= tf.stack([hm_indices[:,0], seq_ids], axis = -1, name = 'scatter_ind')
    fcn_scores_by_class = tf.scatter_nd(pt2_ind, fcn_scores_dense, 
                                        [batch_size, num_classes, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_scores')
    fcn_scores_by_image = tf.scatter_nd(scatter_ind, fcn_scores_dense, 
                                        [batch_size, detections_per_image, fcn_scores_dense.shape[-1]], name='fcn_scores')
    print('    seq_ids                : ', KB.int_shape(seq_ids) ,' Keras tensor ', KB.is_keras_tensor(seq_ids) )  
    print('    sscatter_ids           : ', KB.int_shape(scatter_ind),' Keras tensor ', KB.is_keras_tensor(scatter_ind) )  
    print('    fcn_scores_by_class    : ', KB.int_shape(fcn_scores_by_class) ,' Keras tensor ', KB.is_keras_tensor(fcn_scores_by_class) )  
    print('    fcn_scores_by_image    : ', KB.int_shape(fcn_scores_by_image) ,' Keras tensor ', KB.is_keras_tensor(fcn_scores_by_image) )  
    print('    complete')    
   
    return fcn_scores_by_class, fcn_scores_by_image, pt2_heatmaps, pt2_dense

### Develop `score_fcn_heatmaps()`

In [ ]:
##--------------------------------------------------------------------------
## setup input values
# in_tensor = tf.placeholder(tf.float32, shape=[3,4,32,6], name = 'in_tensor')
# gt_scores  = tf.identity(model_gt_heatmap_scores)
fcn_hm        = tf.identity(model_fcn_heatmap)
pr_hm_scores  = tf.identity(model_pr_heatmap_scores)
in_heatmap    = tf.identity(model_fcn_heatmap)
# config = fcn_model.config
# self   = fcn_model
# names = ['Dev']

fcn_heatmap_scores   = fcn_scoring_graph([fcn_hm , pr_hm_scores], fcn_model.config) 


### evaluate various nodes from computation graph

In [ ]:
sess = KB.get_session()
with sess.as_default():   
#     r_cy = cy.eval()
#     r_cx = cx.eval()
#     r_covar = covar.eval()
#     r_pt2_sum = pt2_sum.eval()
#     r_pt2_ind = pt2_ind.eval()
#     r_pt2_dense = pt2_dense.eval()
#     r_scores = scores.eval()
#     r_both_scores = both_scores.eval()
#     r_normalizer = normalizer.eval()
#     r_scores_norm = scores_norm.eval()print(r_normalizer)
#     r_fcn_scores = fcn_scores.eval()
#     r_Padding = Padding.eval()
#     r_scatter_ind = scatter_ind.eval()
#     r_map_output = map_output.eval()
#     r_fcn_scores_dense = fcn_scores_dense.eval()    
#     r_in_heatmap_norm = in_heatmap_norm.eval()
#     r_fcn_scores = fcn_scores.eval()
    r_fcn_heatmap_scores = fcn_heatmap_scores.eval()    

#     r_in_heatmap  = model_fcn_heatmap
#     r_pt2_hm_reshape = pt2_hm_reshape.eval()    
#     r_hm_indices = hm_indices.eval()
#     r_pt2_heatmaps = pt2_heatmaps.eval()
#     r_pt2_hm_reshape = pt2_hm_reshape.eval()
#     r_hm_indices = hm_indices.eval()
#     r_pt2_heatmaps = pt2_heatmaps.eval()
# print(r_pt2_sum.shape)
# print(r_pt2_ind.shape)
# print(r_pt2_dense.shape)
# print(r_pt2_heatmaps.shape)

# print(pr_hm_scores.shape)
# print(r_fcn_scores.shape)
# print(r_in_heatmap_norm.shape)
print(r_fcn_heatmap_scores.shape)

### Display evaluated items 

In [ ]:
float_formatter = lambda x: "%9.4f" % x
np_format = {}
np_format['float']=float_formatter
np.set_printoptions(linewidth=200, precision=3, floatmode='fixed', threshold =10000, formatter = np_format) 

In [ ]:
  
# print(model_pr_heatmap_scores[0,0,:50])
print('Image ids: ', outputs['mrcnn_input'][1][:,0])
# print(' pt2_dense  :', r_pt2_dense.shape)
# print(r_pt2_dense)
img_id = 0
gt_class_ids = np.unique(model_gt_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
pr_class_ids = np.unique(model_pr_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
fcn_class_ids = np.unique(r_fcn_heatmap_scores[img_id,:,:,4]).astype(int).tolist()
print('Image : {}  GT ClassIds: {}'.format(img_id, gt_class_ids))
print('Image : {}  PR ClassIds: {}'.format(img_id, pr_class_ids))
print('Image : {}  FCN ClassIds: {}'.format(img_id, fcn_class_ids))

In [ ]:
# # print(model_pr_heatmap_scores[0,1,:10])
# for i in range(r_pt2_dense.shape[0]):
#     print(i,r_pt2_dense[i,[4,5,6]], np.sum(r_pt2_heatmaps[i]))

#### display cx,cy,covar

In [ ]:
# print(' cx / cy/ covar:', r_cy.shape, r_cy.shape, r_covar.shape)
# r_xy = np.stack([r_cx,r_cy, ], axis = 1 )
# r_xy_covar = np.concatenate([r_xy, r_covar, r_map_output], axis = 1)
# print(r_cx[12], r_cy[12], r_covar[12,:])

# print('\n map_output')
# print(r_map_output[:20])

# print('\n r_xy_covar')
# print(r_xy_covar[:60])

# print('fcn_scores_dense')
# print(r_fcn_scores_dense)
# print(r_xy_covar)
#     print('    hm_indices :',  r_hm_indices.shape, r_hm_indices.ndim)
#     print(r_hm_indices)
#     print('    pt2_heatmaps.shape: ',r_pt2_heatmaps.shape)
# print(' pt2_ind  : ', r_pt2_ind.shape)    
# print(r_pt2_ind)    
# print(r_scores_norm.shape)
# print(r_scores_norm)    
 
# print(' fcn_scores_dense : ', r_fcn_scores_dense.shape)
# print(r_fcn_scores_dense)
# print(' fcn_scores_by_image :', r_fcn_scores_by_image.shape)
# print(r_fcn_scores_by_image[0,:,4:-4])
print()
# print(r_fcn_scores_by_class[img_id,pr_class_ids,:20,:-8])
 

####  First

In [ ]:
float_formatter = lambda x: "%9.4f" % x
np_format = {}
np_format['float']=float_formatter
np.set_printoptions(linewidth=190, precision=3, floatmode='fixed', threshold =10000, formatter = np_format)
img_id = 0
print('Image : {}  PR ClassIds: {}'.format(img_id, pr_class_ids))
print('Image : {}  fcn ClassIds: {}'.format(img_id, fcn_class_ids))
print(model_fcn_scores.shape)
print(model_pr_heatmap_scores.shape)
for i in  fcn_class_ids:
    if i == 0 :
        continue
    print()
    print('class:', i, '-', dataset_train.class_names[i],'predicted in MRCNN' if i in pr_class_ids else ' ' , ' (Ground Truth) ' if  i in gt_class_ids else ' ')
    for j in range(200):
        print(model_pr_heatmap_scores[img_id, i, j, [4,5,6,7, 11,12,13, 14,15,16, 17,18,19, 20,21,22]])
        print(  r_fcn_heatmap_scores[img_id, i, j,  [4,5,6,7, 11,12,13, 14,15,16, 17,18,19, 20,21,22]])

        print()
        if (np.all( r_fcn_heatmap_scores[img_id,i,j,:4] == 0)):
            break        

#### Second

In [ ]:
img_id = 0 
for i in  pr_class_ids:
    if i == 0 :
        continue
    print()
    print('class:', i, '-', dataset_train.class_names[i],'predicted in MRCNN' if i in pr_class_ids else ' ' , ' (Ground Truth) ' if  i in gt_class_ids else ' ')
    for j in range(200):
#         print('{:>86s} {}  '.format('PR bbox :  ',  r_pr_heatmap_scores[img_id, i,j,:4]))        
        print('{:>86s} {}          {}'.format('class, score, seqIid, norm_score:  ',  model_pr_heatmap_scores[img_id, i,j,[4,6]],model_pr_heatmap_scores[img_id, i,j,[5,7]]))
#         print('{:>86s} {}'.format(' model_pr old style_scores:  ',model_pr_heatmap_scores[img_id, i,j,[8,9,10]]))
#         print('{:>86s} {}'.format(' fcn old style_scores:  ',  r_fcn_heatmap_scores[img_id, i,j,[8,9,10]]))
        print('{:>86s} {}'.format(' model_pr alt_scores_1:  ', model_pr_heatmap_scores[img_id, i,j,11:14]))
        print('{:>86s} {}'.format(' model_pr alt_scores_1 normalized:  ', model_pr_heatmap_scores[img_id, i,j, 14:17]))
        print('{:>86s} {}'.format(' model_pr alt_scores_2:  ',model_pr_heatmap_scores[img_id, i,j, 17:20]))
        print('{:>86s} {}'.format(' model_pr alt_scores_2 normalized:  ',model_pr_heatmap_scores[img_id, i,j,20:23]))
        print()
        print('{:>86s} {}'.format(' fcn alt_scores_1:  '     , r_fcn_heatmap_scores[img_id, i,j,11:14]))
        print('{:>86s} {}'.format(' fcn alt_scores_1 normalized:  '     , r_fcn_heatmap_scores[img_id, i,j,14:17]))    
        print('{:>86s} {}'.format(' fcn alt_scores_2:  ',r_fcn_heatmap_scores[img_id, i,j,17:20]))
        print('{:>86s} {}'.format(' fcn alt_scores_2 normalized:  ',r_fcn_heatmap_scores[img_id, i,j,20:23]))        
        print()
        
        
#         print('{:>86s} {}  '.format('gt bbox :  ',  r_gt_heatmap_scores[img_id, i,j,:4]))
#         print('{:>86s} {}  '.format('gt class, score, seqIid, norm_score:  ',  r_gt_heatmap_scores[img_id, i,j,[4,5,6,7]]))
#         print('{:>86s} {}'.format(' gt old style_scores:  ',model_gt_heatmap_scores[img_id, i,j,[8,9,10]]))
#         print('{:>86s} {}'.format(' gt old style_scores:  ', r_gt_heatmap_scores[img_id, i,j,[8,9,10]]))
#         print('{:>86s} {}'.format(' model gt alt_scores_1:  ',model_gt_heatmap_scores[img_id, i,j,[11,12,13,14,15]]))
#         print('{:>86s} {}'.format(' gt alt_scores_1:  ',r_gt_heatmap_scores[img_id, i,j,[11,12,13,14,15]]))
#         print('{:>86s} {}'.format(' model gt alt_scores_1 normalized:  ',model_gt_heatmap_scores[img_id, i,j,[16,17,18,19,20]]))
#         print('{:>86s} {}'.format(' gt alt_scores_1 normalized:  ',r_gt_heatmap_scores[img_id, i,j,[16,17,18,19,20]]))
#         print('{:>86s} {}'.format(' model gt alt_scores_2::  ',model_gt_heatmap_scores[img_id, i,j, 21:26]))
#         print('{:>86s} {}'.format(' gt alt_scores_2:  ',r_gt_heatmap_scores[img_id, i,j,21:26]))
#         print('{:>86s} {}'.format(' model gt alt_scores_2 normalized:  ',model_gt_heatmap_scores[img_id, i,j,26:31]))
#         print('{:>86s} {}'.format(' gt alt_scores_2 normalized:  ',r_gt_heatmap_scores[img_id, i,j,26:31]))        
        print()
        if (np.all( r_fcn_heatmap_scores[img_id,i,j,:4] == 0)):
            print('break')
            break

#### Third

In [ ]:
float_formatter = lambda x: "%9.4f" % x
np_format = {}
np_format['float']=float_formatter
np.set_printoptions(linewidth=190, precision=3, floatmode='fixed', threshold =10000, formatter = np_format)
img_id = 0
print('Image : {}  PR ClassIds: {}'.format(img_id, pr_class_ids))
print('Image : {}  fcn ClassIds: {}'.format(img_id, fcn_class_ids))
print(model_fcn_scores.shape)
print(model_pr_heatmap_scores.shape)
for i in  fcn_class_ids:
    if i == 0 :
        continue
    print()
    print('class:', i, '-', dataset_train.class_names[i],'predicted in MRCNN' if i in pr_class_ids else ' ' , ' (Ground Truth) ' if  i in gt_class_ids else ' ')
    for j in range(200):
#         print(r_fcn_heatmap_scores[img_id, i, j,[4,5,6,15,16,17,18,19,20,21,22]])
#         print(model_fcn_scores[img_id,i, j, 4:16])
#         print(r_fcn_heatmap_scores[img_id, i, j, 4:16])
#         print()

#         print('\t\t',model_fcn_scores[img_id,i, j, 16:])
#         print('\t\t',r_fcn_heatmap_scores[img_id, i, j, 16:])
#         print()
#         print(model_pr_heatmap_scores[img_id, i, j , [5,6,7, 11,12,13,14,15]])
#         print(r_fcn_heatmap_scores[img_id, i, j,    [5,6,7, 11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]])
        print(model_pr_heatmap_scores[img_id, i, j, [4,5,6,7]])
        print(  r_fcn_heatmap_scores[img_id, i, j, [4,5,6,7]])
        print('\t\t   model alt_scores_1 ',model_pr_heatmap_scores[img_id, i, j, 11:14], '\t\t  norm: ', model_pr_heatmap_scores[img_id, i, j, 14:17])
        print('\t\t     fcn alt_scores_1 ',r_fcn_heatmap_scores[img_id, i, j, 11:14], '\t\t  norm: ',r_fcn_heatmap_scores[img_id, i, j, 14:17])
        print()
        print('\t\t   model alt_scores_2 ',model_pr_heatmap_scores[img_id, i, j, 17:20], '\t\t  norm: ', model_pr_heatmap_scores[img_id, i, j, 20:])
        print('\t\t     fcn alt_scores_2 ',r_fcn_heatmap_scores[img_id, i, j, 17:20], '\t\t  norm: ',r_fcn_heatmap_scores[img_id, i, j, 20:])
        print()
        if (np.all(r_fcn_heatmap_scores[img_id,i,j,:4] == 0)):
            break

In [ ]:
# print(r_in_heatmap.shape)
print( r_pt2_heatmaps.shape) ##,  r_pt2_hm_reshape.shape)
print('pt2_ind')
print(r_pt2_ind.shape)
print(r_pt2_ind)
print('hm_indices')
print(r_hm_indices.shape)
print(r_hm_indices)
print('hm_indices2')
print(r_hm_indices2.shape)
print(r_hm_indices2)
#     for idx in range(178):
#         cls = hm_indices[idx]
#         a = tf.reduce_all(tf.equal(pt2_heatmaps[:,:,idx], in_heatmap[:,:,cls]))
#         print(idx, cls.eval(), a.eval() ) # '  ', a.eval())
## same thing in numpy-------------------
p1_sum = np.sum(r_pt2_heatmaps,axis=(1,2))
p2_sum = np.sum(r_in_heatmap,axis=(1,2))
print(p1_sum.shape, p2_sum.shape)
# print(p2_sum)
print('len(r_hm_indices): ',len(r_hm_indices))
for idx in range(len(r_hm_indices)):
    img = r_hm_indices2[idx,0]
    cls = r_hm_indices2[idx,1]
    print(idx, 'img: ', img, ' cls: ', cls, '   ') ##,r_pt2_ind[idx], r_hm_indices2[idx], r_hm_indices[idx])
    a = np.all(r_pt2_heatmaps[idx,:,:] == r_in_heatmap[img,:,:,cls])
    #     a = np.all(r_pt2_heatmaps[:,:,idx] == r_in_heatmap[:,:,cls])
    print(idx, 'img: ', img, ' cls: ', cls, '   ', a ,'       ',p1_sum[idx], p2_sum[img,cls]) # '  ', a.eval())        

In [ ]:

# img = 1
# for row in range(200):
#     a = np.all(r_fcn_scores_by_image[img,row] == model_fcn_scores[img,row])
#     print(idx, 'img: ', img, ' row:', row, '  a:', a) ##,r_pt2_ind[idx], r_hm_indices2[idx], r_hm_indices[idx])
    
for idx in range(len(r_pt2_ind)):    
    img = r_pt2_ind[idx,0]
    cls = r_pt2_ind[idx,1]
    row = r_pt2_ind[idx,2]
    a = np.all(r_fcn_scores_by_[img,cls,row,:-2] == model_pr_heatmap_scores[img,cls,row])
    print(idx, 'img: ', img, ' cls: ', cls, ' row:', row, '  a:', a) ##,r_pt2_ind[idx], r_hm_indices2[idx], r_hm_indices[idx])
#     print(r_fcn_padded_scores[img,cls,row,:11] )
#     print(model_pr_heatmap_scores[img,cls,row])

    #     a = np.all(r_pt2_heatmaps[:,:,idx] == r_in_heatmap[:,:,cls])
#     print(idx, 'img: ', img, ' cls: ', cls, '   ', a ,'       ',p1_sum[idx], p2_sum[img,cls]) # '  ', a.eval())        

In [ ]:
print(self.config.DETECTION_MAX_INSTANCES)
print(mrcnn_model.config.DETECTION_MAX_INSTANCES)
print(fcn_model.config.DETECTION_MAX_INSTANCES)

print(self.config.DETECTION_PER_CLASS)
print(mrcnn_model.config.DETECTION_PER_CLASS)
print(fcn_model.config.DETECTION_PER_CLASS)

print(mrcnn_model.config.TRAIN_ROIS_PER_IMAGE)
print(fcn_model.config.TRAIN_ROIS_PER_IMAGE)
print(self.config.TRAIN_ROIS_PER_IMAGE)

float_formatter = lambda x: "%10.5f" % x
np_format = {}
np_format['float']=float_formatter
np.set_printoptions(linewidth=150, precision=3, floatmode='fixed', threshold =10000, formatter = np_format)

print(' fcn_scores: ', r_fcn_scores.shape)
print(' padded_fcn_scores: ', r_fcn_padded_scores.shape)
print(' P :', r_P)
print(r_fcn_padded_scores)

print(gt_class_ids)
print(pr_class_ids)


####  Display for visual check - `gauss_scores`  the final result from    `build_heatmap()`

In [ ]:
with sess.as_default():
#     scr_norm     = bbox_scores[...,-1]/ tf.reduce_max(bbox_scores[...,-1], axis = -1, keepdims=True)
#     scr_norm     = tf.where(tf.is_nan(scr_norm),  tf.zeros_like(scr_norm), scr_norm)     
    
    
#     reduce_max = tf.reduce_max(bbox_scores[...,-1], axis = -1, keepdims=True)
#     reduce_min = tf.reduce_min(bbox_scores[...,-1], axis = -1, keepdims=True)  ## epsilon    = tf.ones_like(reduce_max) * 1e-7
#     reduce_max = tf.where(tf.equal(reduce_max, 0.0), epsilon, reduce_max)
#     scr_norm1  = (2* (bbox_scores[...,-1] - reduce_min) / (reduce_max - reduce_min)) - 1
#     scr_norm   = tf.expand_dims(scr_norm, axis = -1)   
    
    res = fcn_scores.eval()
#     bbx =  bbox_scores[...,-1].eval()
#     scr =  scr_norm.eval()
#     scr1 = scr_norm1.eval()

In [ ]:
np.set_printoptions(linewidth=180, precision=5, suppress=False, threshold = 10000)
# print(scr_norm.shape, scr_norm1.shape)
# for i in range(5):
#     for j in range(4):
#         print('bbox')
#         print(bbx[i,j])
#         print('scr_norm')
#         print(scr[i,j])
#         print('scr_norm1')
#         print(scr1[i,j])
print(res.shape)
print(res[...,8:])    

In [ ]:
print(test.shape)
print(pred_heatmap_scores.shape)
# pred_heatmap_scores[0,0]

In [ ]:
np.set_printoptions(linewidth=150, precision=8)
with sess.as_default():
    img = 0
    for k in range(4):
        print('FCN Scroes Image ', img , '/ Class ',k,' ------------')
        print(test[img,k,:,6:])


In [ ]:
np.set_printoptions(linewidth=150, precision=7)
with sess.as_default():
    test1 = tf.reduce_max(fcn_scores[...,-1], axis=-1)
    print(test1.eval())
    epsilon = tf.ones_like(test1) * 1e-7
    print(epsilon.eval())
    test2 = tf.where(tf.equal(test1, 0.0), epsilon, test1)
    print(test2.eval())